## 임솔이

## hate-speech data를 이용하여 적절히 선처리 하여 Transformer를 이용하여 classification
- huggingface에서 사용가능한 여러 한국어 모델을 사용할 것.
- kr-electra모델을 사용할 경우는    
model = ElectraForSequenceClassification.from_pretrained('snunlp/KR-ELECTRA-discriminator')
- 각자 알아서 적절히 한국어 데이터를 preprocessing 할 필요
- 아래에 주어진 cell에 따라 해도 되나, 자기의 방식대로 할 수 있음. 단 셀이나 코드에 필요한 설명을 주석처리하여 보여 줄 것. 채점시 의도를 파악할 수 있도록 
- 주피터노트북 파일명에 각자이름 붙일것: hsp_classification_FinTechTest_각자이름.ipynb
- 제출은 조교에게 (김은진( jyej3154@snu.ac.kr)), 메일제목은 Fintech_NLP_test_각자이름

## Download Dataset

In [ ]:
## 데이터 다운로드
#다운로드 받은 데이터는 이 코드와 같은 디렉토리에 있는 것으로 하여 코딩
!wget -nc https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv
!wget -nc https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/dev.tsv

--2022-04-28 08:49:55--  https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 913546 (892K) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>] 892.13K  --.-KB/s    in 0.04s   

2022-04-28 08:49:55 (19.9 MB/s) - ‘train.tsv’ saved [913546/913546]

--2022-04-28 08:49:55--  https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54903 (54K) [text/p

In [ ]:
!nvidia-smi

Thu Apr 28 08:49:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Packages

In [ ]:
## 필요한 패키지 설치 예시 
!pip install transformers
!pip install --upgrade keras
!pip install --upgrade tensorflow

!pip install --ignore-installed hanja

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.6 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 462 kB 5.3 MB/s 
     |████████████████████████████████| 120 kB 5.4 MB/s 
     |████████████████████████████████| 265 kB 44.1 MB/s 
     |████████████████████████████████| 297 kB 44.1 MB/s 
     |████████████████████████████████| 207 kB 46.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 61 kB 8.1 MB/s 
     |████████████████████████████████| 138 kB 51.0 MB/s 
     |████████████████████████████████| 149 kB 48.9 MB/s 
     |████████████████████████████████| 98 kB 6.

# 필요한 모둘 로드

In [ ]:
import os
import torch
import numpy as np
import pandas as pd

from transformers import Trainer, TrainingArguments
from transformers import ElectraForSequenceClassification, ElectraTokenizer

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import seaborn as sn

## Load hate Data & Process

### 원 데이터

![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-04-27%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.57.01.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-04-27%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.57.01.png)

### 본 과제에서는 hate label만 사용

![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-04-27%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.59.20.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-04-27%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.59.20.png))

In [ ]:
!head train.tsv

comments	contain_gender_bias	bias	hate
(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....	False	others	hate
....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ	False	none	none
...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래야, 공정한 사회지...심은대로 거두거라...	False	none	hate
1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데	False	none	none
1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노	True	gender	hate
10+8 진짜 이승기랑 비교된다	False	none	none
100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.	True	gender	hate
10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음	False	none	none
10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐	False	none	offensive


In [ ]:
!head dev.tsv

comments	contain_gender_bias	bias	hate
송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.	False	none	none
지현우 나쁜놈	False	none	offensive
알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라	False	none	hate
설마 ㅈ 현정 작가 아니지??	True	gender	hate
이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀 하고사시죠.	False	none	offensive
일베충들 ㅂㄷ거리는것봐라 ㅉㅉ	False	others	hate
아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다.	False	none	none
강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요	False	none	none
알았어 그만	False	none	offensive


In [ ]:
#Load and Convert Data
test_dataset = pd.read_csv('dev.tsv', sep='\t', quoting=3) # quoting=3: 큰따옴표 무시
train_dataset = pd.read_csv('train.tsv', sep='\t', quoting=3)

print(len(train_dataset), len(test_dataset))

7896 471


In [ ]:
# 원데이터
train_dataset.head()

,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate


In [ ]:
# 본 과제에서는 hate label만 사용 (comments-hate)
train_dataset = train_dataset[['comments' ,'hate']]
test_dataset = test_dataset[['comments' ,'hate']]

In [ ]:
train_dataset.head()

,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [ ]:
test_dataset.head()

,comments,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive


## Preprocessing

In [ ]:
train_dataset['hate'].unique()

array(['hate', 'none', 'offensive'], dtype=object)

In [ ]:
train_dataset['hate'].replace(['hate', 'none', 'offensive'],[0,1,2], inplace=True) 

In [ ]:
test_dataset['hate'].replace(['hate', 'none', 'offensive'],[0,1,2], inplace=True) 

In [ ]:
train_dataset.rename(columns = {'hate' : 'label'}, inplace = True)
test_dataset.rename(columns = {'hate' : 'label'}, inplace = True)

In [ ]:
train_dataset.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,0
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,1
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",0
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",1
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,0


## Build Dataset

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("snunlp/KR-ELECTRA-discriminator", num_labels=3)

Downloading:   0%|          | 0.00/209k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468 [00:00<?, ?B/s]

In [ ]:
lens = []
for doc in train_dataset['comments']:
  lens.append(len(tokenizer.tokenize(doc)))
print(max(lens), sum(lens)/len(lens), min(lens))

80 22.55851063829787 2


In [ ]:
#Make Dataset Class
class label_data(torch.utils.data.Dataset): #torch dataset으로 감싸줌.
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.label = torch.tensor(dataset['label']).long() # label label 가져오기!!
        features = tokenizer([str(x) for x in dataset.comments], padding=True, truncation=True, max_length=128) #list에 features를 불러온다.
        self.input = torch.tensor(features['input_ids'])
        self.mask = torch.tensor(features['attention_mask'])
    
    def __len__(self):
        return len(self.input)

    def __getitem__(self, index): # 각각 어떤 값들을 가져올지 dictoinary로 만들어준다.
        return {'input_ids': self.input[index], 'attention_mask': self.mask[index], 'label':self.label[index]}

In [ ]:
train_data = label_data(train_dataset, tokenizer)
test_data = label_data(test_dataset, tokenizer)

In [ ]:
train_data[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    2,    12,  8553,  9975,  5186,  5208, 12682,    13,  3556, 21093,
          2405, 15052, 21326,  5187,  2407, 24398,  5640,  5036,  9975,  5066,
          5105,  5383,  5042,  6489, 11511,  9874,  8501, 13363,  5418,  5018,
            18,    18,    18,    18,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'label': tensor(0)}

In [ ]:
# check
train_dataset.iloc[0]

comments    (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...
label                                                       0
Name: 0, dtype: object

In [ ]:
tokenizer.decode(train_data[0]['input_ids'])

'[CLS] ( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네.... [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

## Load Model

In [ ]:
model = ElectraForSequenceClassification.from_pretrained('snunlp/KR-ELECTRA-discriminator', num_labels=3)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at snunlp/KR-ELECTRA-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-ELECTRA-discriminator and are newly initialized: ['classifier.dense

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids # 정답!! 
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro') # 'hate', 'none', 'offensive'

  # 'micro': 각 샘플이나 예측에 동일한 가중치를 부여하고자 할 때 (accuracy)
  # 'macro': 모든 클래스에 동일한 가중치를 부여
  # 'weighted': 각 레이블이 불균형할 때 weight를 주어 계산. precision과 recall의 합이 아닌 F-score를 야기할 수 있음.
  # https://towardsdatascience.com/micro-macro-weighted-averages-of-f1-score-clearly-explained-b603420b292f#989c



    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
  #model.to_device > 굳이 안해줘도 hugging face를 사용하면 알아서 올려줌.

## Parameter setting

In [ ]:
#Trainer
#Hyper parameters
training_args = TrainingArguments(
    output_dir='./results', # 내가 훈련한 모델을 저장할 directory
    num_train_epochs=3,  # 몇 epoch를 돌릴건지
    per_device_train_batch_size=64, # decrease batch size to avoid OOM error, train, test batch size, out of memory 발생했을때, batch size를 가장먼저 줄여주면 좋음. batch가 크면 한번에 계산하는 것이므로 메모리가 부족할 수가 있음
    per_device_eval_batch_size=128, # 기본은 8, 8 왜 batch size를 지금 다르게 해줬나? eval은 점수만 받으면 되므로 batch size를 크게준것, train은 중요하니깐.

    warmup_steps=500,
    save_steps=200, # 200번째,400번째 돈 모델이 저장된다.
    
    logging_dir='./logs',

    # learning_rate=2e-5, 5e-5(기본)
    weight_decay=0.01,

    # evaluation_strategy="steps",
    # load_best_model_at_end = True
    # metric_for_best_model="accuracy",
    # dataloader_pin_memory=False, 
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)

## Train

In [ ]:
#Train Model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7896
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 372


Step,Training Loss


Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=372, training_loss=0.8050682313980595, metrics={'train_runtime': 601.2762, 'train_samples_per_second': 39.396, 'train_steps_per_second': 0.619, 'total_flos': 998194750799904.0, 'train_loss': 0.8050682313980595, 'epoch': 3.0})

## Evaluate

In [ ]:
#Evaluate Model
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 471
  Batch size = 128


{'epoch': 3.0,
 'eval_accuracy': 0.6963906581740976,
 'eval_f1': 0.6978773675175591,
 'eval_loss': 0.7119706273078918,
 'eval_precision': 0.7110973545122427,
 'eval_recall': 0.7097493277821147,
 'eval_runtime': 3.8501,
 'eval_samples_per_second': 122.333,
 'eval_steps_per_second': 1.039}